In [8]:
%run ../../../utils/commonImports.py
%run ../../../utils/tradingImports.py
%matplotlib inline

from base import supres
from exputils import *

# Load data

In [9]:
# dropbox_dir = 'D:\\Dropbox\\My work\\krypl-project'
dropbox_dir = '/Users/david.vesely/Dropbox/My work/krypl-project'
features = ['support']
pairs = read_tsv('selected-pairs.tsv', header=None)[0].tolist()

data_dict = {p: read_all(p, dropbox_dir) for p in pairs}
supports = read_tsv('labeled.tsv')

In [10]:
window_sizes = [4]
shifts = [1, 2, 3]

DAY = 48
max_hold_time = 5 * DAY
stop_loss = 0.05
fee = 0.0025

# Support changes

In [11]:
def get_changes(data, shift=1):
    support_vals = data['support'].values
    size = support_vals.shape[0]
    changes = (support_vals[shift:] / support_vals[:size-shift]) - 1
    for _ in range(shift):
        changes = np.insert(changes, 0, 0)
    data[f'change_{shift}'] = changes
    return data

In [12]:
from tqdm import tqdm_notebook

n = len(pairs) * len(window_sizes) * len(shifts)
with tqdm_notebook(total=n) as pbar:
    supports_data = pd.DataFrame()
    for pair in pairs:
        for win_size in window_sizes:
            supports_df = supports.query(f'pair == "{pair}"')\
                    .query(f'win_size == {win_size}')\
                    .sort_values('date')\
                    .copy()
            for shift in shifts:
                supports_df = get_changes(supports_df, shift)
                pbar.update()
                
            supports_data = supports_data.append(supports_df)
            

# Consolidate label for better learning

In [19]:
def decision(r):
    if r['max_profit'] > 0 and r['max_profit'] >= -r['change_1']*0.85:
        return abs(r['change_1'])*0.85
    else:
        return r['max_profit']

supports_data['max_profit_consolidated'] = supports_data.apply(lambda r: decision(r), axis=1)

In [20]:
write_tsv(supports_data, 'featured.tsv')

In [21]:
supports_data

,index,support,pair,win_size,date,max_profit,change_1,change_2,change_3,max_profit_consolidated
0,24.0000,0.0021,BTC_ETH,4,2016-01-01 12:00:00,0.0261,0.0000,0.0000,0.0000,0.0000
1,33.0000,0.0022,BTC_ETH,4,2016-01-01 16:30:00,-0.0040,0.0089,0.0000,0.0000,-0.0040
2,45.0000,0.0022,BTC_ETH,4,2016-01-01 22:30:00,0.0274,-0.0015,0.0074,0.0000,0.0013
3,58.0000,0.0022,BTC_ETH,4,2016-01-02 05:00:00,0.0079,0.0234,0.0218,0.0309,0.0199
4,70.0000,0.0022,BTC_ETH,4,2016-01-02 11:00:00,0.0020,0.0034,0.0269,0.0253,0.0029
5,84.0000,0.0022,BTC_ETH,4,2016-01-02 18:00:00,0.0071,-0.0173,-0.0140,0.0091,0.0071
6,94.0000,0.0022,BTC_ETH,4,2016-01-02 23:00:00,0.0035,-0.0085,-0.0257,-0.0224,0.0035
7,99.0000,0.0022,BTC_ETH,4,2016-01-03 01:30:00,0.0102,0.0007,-0.0078,-0.0250,0.0006
8,107.0000,0.0022,BTC_ETH,4,2016-01-03 05:30:00,-0.0002,0.0135,0.0143,0.0056,-0.0002
9,112.0000,0.0022,BTC_ETH,4,2016-01-03 08:00:00,0.0524,0.0010,0.0145,0.0153,0.0008
